In [1]:
from greensight.utils import DIR_ROOT, DIR_DATA
import pandas as pd
import numpy as np
import json
import pandas as pd

In [2]:
# load DataFrame
s1_path = DIR_DATA / "processed_data/sentinel_one_df.hdf"

df_data = pd.read_hdf(s1_path, key="df")
df_orig = df_data.copy()
df_data

greenbelt  Merseyside and Greater Manchester                                   \
shape                              E06000006                                    
band                                   VHAsc      VHDes      VVAsc      VVDes   
2016-01-01                        -21.880295 -16.991904 -15.775903  -9.790099   
2016-02-01                        -20.987328 -17.073966 -14.334226  -9.872628   
2016-03-01                        -21.244966 -17.481998 -14.993133 -10.083542   
2016-04-01                        -23.155242 -17.578793 -16.930790  -9.725071   
2016-05-01                        -23.382046 -18.103710 -17.087475 -10.531700   
...                                      ...        ...        ...        ...   
2024-06-01                        -17.575681 -18.301998 -10.910621 -11.432426   
2024-07-01                        -17.042238 -18.078907 -10.479614 -11.432295   
2024-08-01                        -17.215400 -18.273017 -10.457908 -11.061045   
2024-09-01                        -17.063121 -17.733257 -10.062128 -10.583111   
2024-10-01                        -17.428285 -17.976361 -10.137424  -9.999138   

greenbelt                                                                     \
shape       E06000007                                   E06000008              
band            VHAsc      VHDes      VVAsc      VVDes      VHAsc      VHDes   
2016-01-01 -16.071707 -16.330632  -9.220326  -9.751699 -16.652340 -16.737056   
2016-02-01 -17.203276 -16.601356 -10.011234  -9.807394 -18.008646 -17.417576   
2016-03-01 -16.309546 -16.989151  -9.348067 -10.051062 -17.052957 -17.576936   
2016-04-01 -16.492496 -16.878869  -9.339440  -9.714068 -16.769975 -16.911006   
2016-05-01 -16.863572 -17.473715  -9.546583 -10.452768 -17.058511 -18.111202   
...               ...        ...        ...        ...        ...        ...   
2024-06-01 -17.293008 -17.948754 -10.802252 -11.310022 -18.270138 -18.840724   
2024-07-01 -17.280203 -17.979831 -10.731518 -11.481049 -18.065222 -18.766206   
2024-08-01 -17.276824 -17.965424 -10.439897 -10.900445 -17.807154 -18.618061   
2024-09-01 -17.066896 -17.645074  -9.842672 -10.406478 -17.996418 -18.481467   
2024-10-01 -17.019582 -17.728231  -9.643953  -9.785701 -18.343287 -18.748475   

greenbelt   ...    London                                              \
shape       ... E09000027             E09000029                         
band        ...     VVAsc      VVDes      VHAsc      VHDes      VVAsc   
2016-01-01  ... -8.756936 -10.676782 -16.771370 -17.255673 -10.597951   
2016-02-01  ... -9.768301 -11.732255 -17.869552 -18.684054 -11.592459   
2016-03-01  ... -9.616047 -12.070300 -17.526683 -18.222637 -11.388633   
2016-04-01  ... -8.943721 -11.705665 -17.185180 -17.619796 -10.872856   
2016-05-01  ... -9.011251 -11.404164 -17.095400 -17.859541 -10.820224   
...         ...       ...        ...        ...        ...        ...   
2024-06-01  ... -9.298970 -11.303450 -17.345873 -18.147220 -10.786822   
2024-07-01  ... -8.652883 -11.078900 -17.375320 -17.882205 -10.816706   
2024-08-01  ... -9.483580 -11.416825 -18.281246 -18.453976 -11.732576   
2024-09-01  ... -9.313126 -10.906338 -18.312653 -18.213823 -11.435320   
2024-10-01  ... -8.593788 -11.184389 -17.867922 -18.473817 -10.839669   

greenbelt                                                          
shape                  E09000031                                   
band            VVDes      VHAsc      VHDes      VVAsc      VVDes  
2016-01-01 -11.723700 -16.742193 -17.174760 -10.834776 -11.552687  
2016-02-01 -12.890625 -17.622554 -18.572617 -11.384257 -12.543675  
2016-03-01 -12.614200 -17.260247 -18.026064 -11.268428 -12.471184  
2016-04-01 -11.681797 -16.307043 -17.452870 -10.085572 -11.418674  
2016-05-01 -12.022884 -16.547483 -17.510375 -10.391498 -11.840899  
...               ...        ...        ...        ...        ...  
2024-06-01 -12.055723 -17.763319 -18.518028 -10.941011 -12.222783  
2024-07-01 -11.704129 -17.498436 -18.2617

In [3]:
lookup_path = DIR_ROOT / "resources/id_lookup/id_lookup.json"
with open(lookup_path, "r") as in_file:
    D_lookup = json.load(in_file)

In [4]:
gb_names = list(set([v["GB_Name"] for v in D_lookup.values()]))
len(gb_names)

16

In [5]:
gb_areas = {}

for name in gb_names:
    area = np.sum([v["Area_ha"] for k, v in D_lookup.items() if v["GB_Name"] == name])
    gb_areas[name] = area

In [6]:
# create weighting dataframe

# instantiate empty df
df_weights = pd.DataFrame(np.nan, index=df_data.index, columns=df_data.columns )

# apply weighting
for code, value in D_lookup.items():
    area = value["Area_ha"]
    gb_name = value["GB_Name"]
    greenbelt_area = gb_areas[gb_name]

    df_weights.loc[:, (slice(None), code, slice(None))] = area/greenbelt_area


df_weights.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " "), i[1], i[2]) for i in df_weights.columns])
df_weights.columns.names = ["greenbelt", "shape", "band"]
df_weights.head()

greenbelt  Merseyside and Greater Manchester                                \
shape                              E06000006                                 
band                                   VHAsc     VHDes     VVAsc     VVDes   
2016-01-01                          0.009691  0.009691  0.009691  0.009691   
2016-02-01                          0.009691  0.009691  0.009691  0.009691   
2016-03-01                          0.009691  0.009691  0.009691  0.009691   
2016-04-01                          0.009691  0.009691  0.009691  0.009691   
2016-05-01                          0.009691  0.009691  0.009691  0.009691   

greenbelt                                                               ...  \
shape      E06000007                               E06000008            ...   
band           VHAsc     VHDes     VVAsc     VVDes     VHAsc     VHDes  ...   
2016-01-01  0.053096  0.053096  0.053096  0.053096  0.024632  0.024632  ...   
2016-02-01  0.053096  0.053096  0.053096  0.053096  0.024632  0.024632  ...   
2016-03-01  0.053096  0.053096  0.053096  0.053096  0.024632  0.024632  ...   
2016-04-01  0.053096  0.053096  0.053096  0.053096  0.024632  0.024632  ...   
2016-05-01  0.053096  0.053096  0.053096  0.053096  0.024632  0.024632  ...   

greenbelt     London                                                    \
shape      E09000027           E09000029                                 
band           VVAsc     VVDes     VHAsc     VHDes     VVAsc     VVDes   
2016-01-01  0.000264  0.000264  0.001188  0.001188  0.001188  0.001188   
2016-02-01  0.000264  0.000264  0.001188  0.001188  0.001188  0.001188   
2016-03-01  0.000264  0.000264  0.001188  0.001188  0.001188  0.001188   
2016-04-01  0.000264  0.000264  0.001188  0.001188  0.001188  0.001188   
2016-05-01  0.000264  0.000264  0.001188  0.001188  0.001188  0.001188   

greenbelt                                           
shape      E09000031                                
band           VHAsc     VHDes     VVAsc     VVDes  
2016-01-01  0.001653  0.001653  0.001653  0.001653  
2016-02-01  0.001653  0.001653  0.001653  0.001653  
2016-03-01  0.001653  0.001653  0.001653  0.001653  
2016-04-01  0.001653  0.001653  0.001653  0.001653  
2016-05-01  0.001653  0.001653  0.001653  0.001653  

[5 rows x 720 columns]

In [7]:
D_lookup["E07000197"]

{'GB_Name': 'Stoke-on-Trent', 'Area_ha': 9292.431604283165}

In [8]:
shapes = set([i[0] for i in  list(df_weights["Birmingham"].columns)])
shapes2 = set([code for code, value in D_lookup.items() if value["GB_Name"] == "Birmingham"])

In [11]:
# check all greenbelts for each band sum to 1.0
assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
df_weights.groupby(level=["greenbelt", "band"], axis=1).sum().head()

/tmp/ipykernel_47711/537146442.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
/tmp/ipykernel_47711/537146442.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_weights.groupby(level=["greenbelt", "band"], axis=1).sum().head()


greenbelt  Bath and Bristol                   Birmingham                    \
band                  VHAsc VHDes VVAsc VVDes      VHAsc VHDes VVAsc VVDes   
2016-01-01              1.0   1.0   1.0   1.0        1.0   1.0   1.0   1.0   
2016-02-01              1.0   1.0   1.0   1.0        1.0   1.0   1.0   1.0   
2016-03-01              1.0   1.0   1.0   1.0        1.0   1.0   1.0   1.0   
2016-04-01              1.0   1.0   1.0   1.0        1.0   1.0   1.0   1.0   
2016-05-01              1.0   1.0   1.0   1.0        1.0   1.0   1.0   1.0   

greenbelt  Blackpool        ... Stoke on Trent       Tyne and Wear        \
band           VHAsc VHDes  ...          VVAsc VVDes         VHAsc VHDes   
2016-01-01       1.0   1.0  ...            1.0   1.0           1.0   1.0   
2016-02-01       1.0   1.0  ...            1.0   1.0           1.0   1.0   
2016-03-01       1.0   1.0  ...            1.0   1.0           1.0   1.0   
2016-04-01       1.0   1.0  ...            1.0   1.0           1.0   1.0   
2016-05-01       1.0   1.0  ...            1.0   1.0           1.0   1.0   

greenbelt               York                    
band       VVAsc VVDes VHAsc VHDes VVAsc VVDes  
2016-01-01   1.0   1.0   1.0   1.0   1.0   1.0  
2016-02-01   1.0   1.0   1.0   1.0   1.0   1.0  
2016-03-01   1.0   1.0   1.0   1.0   1.0   1.0  
2016-04-01   1.0   1.0   1.0   1.0   1.0   1.0  
2016-05-01   1.0   1.0   1.0   1.0   1.0   1.0  

[5 rows x 64 columns]

In [12]:
# save DataFrame
output_path = DIR_DATA / "processed_data/sentinel_one_weights.hdf"
df_weights.to_hdf(output_path, key="df", mode="w", format="table")